<a href="https://colab.research.google.com/github/nohandsomewujun/2022python_hw/blob/main/lab/level3.15.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# level3.15 code
本次实验使用了python的faceswap相关工具实现了将视频中自己的脸换成脸钢铁侠的脸。替换的结果在同目录的mp4文件夹下，里面包含了原视频，替换的人脸和替换后的视频。
*因为深度学习自己实现起来比较困难，而且也不是大作业，所以这里就直接用了别人训练好的模型。*
（如果要运行复现的话建议在Google colab里面打开此文件）

## 准备工作（方便助教小哥哥复现代码结果

**一定要确保runtime的设置里面是在GPU下运行的！！！否则可能会出错**

In [ ]:
# 首先应当将这个Github上的项目拉到本地
!git clone https://github.com/neuralchen/SimSwap
!cd SimSwap && git pull

In [ ]:
# 安装相关依赖
!pip install insightface==0.2.1 onnxruntime moviepy
!pip install googledrivedownloader
!pip install imageio==2.4.1

In [ ]:
import os
os.chdir("SimSwap")
!ls

利用Googledrive将别人训练好的模型下载下来

In [ ]:
from google_drive_downloader import GoogleDriveDownloader

!wget -P ./arcface_model https://github.com/neuralchen/SimSwap/releases/download/1.0/arcface_checkpoint.tar
!wget https://github.com/neuralchen/SimSwap/releases/download/1.0/checkpoints.zip
!unzip ./checkpoints.zip  -d ./checkpoints
!wget -P ./parsing_model/checkpoint https://github.com/neuralchen/SimSwap/releases/download/1.0/79999_iter.pth

In [ ]:
!wget --no-check-certificate "https://sh23tw.dm.files.1drv.com/y4mmGiIkNVigkSwOKDcV3nwMJulRGhbtHdkheehR5TArc52UjudUYNXAEvKCii2O5LAmzGCGK6IfleocxuDeoKxDZkNzDRSt4ZUlEt8GlSOpCXAFEkBwaZimtWGDRbpIGpb_pz9Nq5jATBQpezBS6G_UtspWTkgrXHHxhviV2nWy8APPx134zOZrUIbkSF6xnsqzs3uZ_SEX_m9Rey0ykpx9w" -O antelope.zip
!unzip ./antelope.zip -d ./insightface_func/models/

## 利用model开始运算

In [ ]:
import cv2
import torch
import fractions
import numpy as np
from PIL import Image
import torch.nn.functional as F
from torchvision import transforms
from models.models import create_model
from options.test_options import TestOptions
from insightface_func.face_detect_crop_multi import Face_detect_crop
from util.videoswap import video_swap
from util.add_watermark import watermark_image

In [ ]:
transformer = transforms.Compose([
        transforms.ToTensor(),
    ])

transformer_Arcface = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

detransformer = transforms.Compose([
        transforms.Normalize([0, 0, 0], [1/0.229, 1/0.224, 1/0.225]),
        transforms.Normalize([-0.485, -0.456, -0.406], [1, 1, 1])
    ])

In [ ]:
opt = TestOptions()
opt.initialize()
opt.parser.add_argument('-f')
opt = opt.parse()
opt.pic_a_path = './demo_file/Iron_man.jpg'  # 这里为人脸的图片
opt.video_path = './demo_file/3.mp4'  # 这里为你要替换的视频
opt.output_path = './output/3iron.mp4'  # 输出保存路径
opt.temp_path = './tmp'
opt.Arc_path = './arcface_model/arcface_checkpoint.tar'
opt.isTrain = False
opt.use_mask = True

crop_size = opt.crop_size

torch.nn.Module.dump_patches = True
model = create_model(opt)
model.eval()

app = Face_detect_crop(name='antelope', root='./insightface_func/models')
app.prepare(ctx_id=0, det_thresh=0.6, det_size=(640, 640))

with torch.no_grad():
    pic_a = opt.pic_a_path
    img_a_whole = cv2.imread(pic_a)
    img_a_align_crop, _ = app.get(img_a_whole, crop_size)
    img_a_align_crop_pil = Image.fromarray(
        cv2.cvtColor(img_a_align_crop[0], cv2.COLOR_BGR2RGB))
    img_a = transformer_Arcface(img_a_align_crop_pil)
    img_id = img_a.view(-1, img_a.shape[0], img_a.shape[1], img_a.shape[2])
    img_id = img_id.cuda()
    img_id_downsample = F.interpolate(img_id, size=(112, 112))
    latend_id = model.netArc(img_id_downsample)
    latend_id = latend_id.detach().to('cpu')
    latend_id = latend_id/np.linalg.norm(latend_id, axis=1, keepdims=True)
    latend_id = latend_id.to('cuda')

    video_swap(opt.video_path, latend_id, model, app, opt.output_path,
               temp_results_dir=opt.temp_path, use_mask=opt.use_mask)


如果不出意外的话一路点运行应该就可以了（